In [35]:
!pip install -r requirement.txt
import psycopg2
import dateutil.parser
import os 
import sys
import itertools
import matplotlib.pyplot as plt
import database_connection as db_connection
import json
import pandas as pd
import numpy as np

  Using cached matplotlib-3.4.2-cp39-cp39-macosx_10_9_x86_64.whl (7.2 MB)
  Using cached numpy-1.21.0-cp39-cp39-macosx_10_9_x86_64.whl (16.9 MB)
  Using cached pandas-1.2.5-cp39-cp39-macosx_10_9_x86_64.whl (10.7 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.2
    Uninstalling numpy-1.20.2:
      Successfully uninstalled numpy-1.20.2
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.3
    Uninstalling pandas-1.2.3:
      Successfully uninstalled pandas-1.2.3
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.4.1
    Uninstalling matplotlib-3.4.1:
      Successfully uninstalled matplotlib-3.4.1


In [2]:
request  = """SELECT * FROM partner_alma_data.payload_pending_stat
ORDER BY equipment_id ASC"""
df = db_connection.send_request((request))
field='c28'

In [3]:
df_value_equipment_id = pd.DataFrame(df[['equipment_id','value']])

In [4]:
df_value_equipment_id

,equipment_id,value
0,1,"{""t"":""2021-06-08T13:24:54"",""lat"":45.53691,""lng..."
1,1,"{""t"":""2021-06-08T13:31:24"",""lat"":45.47813,""lng..."
2,1,"{""t"":""2021-06-08T13:34:54"",""lat"":45.46051,""lng..."
3,1,"{""t"":""2021-06-08T13:28:54"",""lat"":45.49832,""lng..."
4,1,"{""t"":""2021-06-08T13:25:24"",""lat"":45.53207,""lng..."
...,...,...
2634978,340,"{""t"":""2021-05-27T23:08:42"",""c4_0821"":31.00,""la..."
2634979,340,"{""t"":""2021-05-27T23:33:28"",""c4_0821"":32.00,""la..."
2634980,340,"{""t"":""2021-05-27T23:38:09"",""c4_0821"":50,""lat"":..."
2634981,340,"{""t"":""2021-05-27T23:38:20"",""c4_0821"":3,""lat"":4..."


**Methodes pour la construction et la recuperation des données**

In [5]:
def get_all_fields(df):
    all_fields = set()
    for val in df.value:
        try:
            all_fields = all_fields.union(set(list(json.loads(val).keys())))
        except:
            pass
    return all_fields
all_fields=get_all_fields(df_value_equipment_id)

In [6]:
def get_sensors_value_eqp_datetime(field):
    
    values_payload_eqp = {element:list() for element in df.equipment_id.unique() }
    for eqp_id in values_payload_eqp.keys():
        temp = df_value_equipment_id.groupby('equipment_id').get_group(eqp_id)
        try:

            payloads = [(json.loads(element).get(field),json.loads(element).get("t")) 
                        for element in temp.value.values if json.loads(element).get(field)]
            values_payload_eqp[eqp_id]= payloads
        except:
            pass
    return values_payload_eqp

In [28]:
def get_by_day(list_tuple):
    groups = []
    uniquekeys = []
    for k, g in itertools.groupby(list_tuple, key=lambda x: x[1].date()):
       uniquekeys.append(k)    
       groups.append(list(g))

    return groups

In [8]:
def get_graph(x,y,title):
    plt.plot(x,y)
    plt.title(title)
    plt.xticks(rotation=45) 
    plt.figure()

In [9]:
def get_consecutive_date(list_date):
    res = list()
    for element in list_date:
        if len(res)==0:
            res.append(element)
        else:
            if abs((res[-1][-1][1]-element[0][1]).days)==1:
                res[-1]=unique_consecutive(res[-1],element)
            else:
                res.append(element)
    return res

In [10]:
def get_graph_by_day(list_tuple):
    to_plot = get_by_day(list_tuple)
    
    for element in to_plot:
        x = [e[1] for e in element ]
        y = [e[0] for e in element]
        title = str(element[0][1].date())
        get_graph(x,y,title)
        

In [11]:
def unique_consecutive(list1,list2):
    res = list()
    for element in list1:
        res.append(element)
    for element in list2:
        res.append(element)
    return res

In [12]:
result = get_sensors_value_eqp_datetime(field)

In [14]:
def get_maxs(offset):
    if offset<=0:
        return
    if offset==1:
        max_temp = sorted([(len(result[key]),key) for key in result.keys()])[-1]
        eqpmax_date = [(element[0],dateutil.parser.parse(element[1])) for element in result[max_temp[-1]]]
        eqpmax_date=sorted(eqpmax_date,key = lambda x:x[1])
        return max_temp[1],eqpmax_date
    else:
        
        max_temp = sorted([(len(result[key]),key) for key in result.keys()])
        max_temp.reverse()
        keys =[ e[1] for e in max_temp[:offset]]
        max_temp = [ e[1] for e in max_temp[:offset]]
        eqpmaxs_dates = list()
        for value in max_temp: ##ici on a la list des id des n equipement ayant remontés le plus de données
            temp = ([(element[0],dateutil.parser.parse(element[1])) for element in result[value]])
            temp = sorted(temp,key = lambda x:x[1])
            eqpmaxs_dates.append(temp)
        return dict(zip(keys,eqpmaxs_dates))

In [15]:
def optimized_offset(dict_values):
    return len([a for a in dict_values.items() if len(a[1])>0])

In [30]:
#result -> len =304
# non vide = 28
#vide = 276
#\/
optimized_offset(result)
max_ = get_maxs(optimized_offset(result))

### print the group of consecutives datetime with values associated ### 
dataframes_per_consecutives_days

In [17]:
dfs = list()
max_ = get_maxs(optimized_offset(result))
for key,value in max_.items(): 
    for element in get_by_day(value):
        temp = [(key,a,b) for a,b in element ]
        dfs.append(pd.DataFrame(data=temp, columns = ['Equipment_id','Value','Date']))
    for d in dfs:
        d['Value']=d['Value'].abs()

#for element in dataframes_per_consecutives_days:
#plt.title(element.Equipment_id.unique())
#plt.plot(element.Date,element.Value)
#plt.xticks(rotation=60)
#plt.show()

In [18]:
dfs

[     Equipment_id  Value                Date
 0              93     39 2021-05-27 15:00:19
 1              93     40 2021-05-27 15:00:31
 2              93     39 2021-05-27 15:01:17
 3              93     40 2021-05-27 15:01:28
 4              93     39 2021-05-27 15:01:40
 ..            ...    ...                 ...
 297            93      8 2021-05-27 23:29:41
 298            93      7 2021-05-27 23:35:15
 299            93      8 2021-05-27 23:37:33
 300            93      9 2021-05-27 23:45:03
 301            93     16 2021-05-27 23:53:06
 
 [302 rows x 3 columns],
      Equipment_id  Value                Date
 0              93     17 2021-05-28 00:02:31
 1              93     18 2021-05-28 00:12:07
 2              93     19 2021-05-28 00:22:06
 3              93     20 2021-05-28 00:33:03
 4              93     21 2021-05-28 00:50:42
 ..            ...    ...                 ...
 154            93     34 2021-05-28 08:24:02
 155            93     35 2021-05-28 09:30:04
 156   

## ça semble ok 

In [20]:
def save_datasets():
    path = os.getcwd()+"/DataFrames/"+str(field)
    choice = input('save ?')
    if(choice in ['yes',',no'] and choice =='yes'):
        for e in dfs:
            key = str(e.Equipment_id.unique()[-1])
            if not os.path.exists(path+"/"+key):
                os.mkdir(path+"/"+key)
                print("Directory " , path+"/"+key ,  " Created ")
            timestamp = str(e.Date[0].date())
            e.to_csv(path_or_buf=path+"/{}/{}.csv".format(key,timestamp))
            print('Dataset saved to '+path+'/'+key)
            #pour la suite on va sur colab -> diminution des temps de traitement et peut etre ML pour la modelisation de la courbe

#### save_datasets()